In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

driver = webdriver.Chrome()
driver.set_window_size(1440, 900)

In [2]:

def defilement(id_element, duree_defilement):
    # Click sur élément pour permettre le défilement
    boite_scroll = driver.find_element(by=By.ID, value=id_element)
    boite_scroll.click()

    # Défilement via simulation touches T inversées (mauvais résultats lorsque simulation d'un défilement à la souris)
    ActionChains(driver).key_down(Keys.ARROW_DOWN).perform()
    time.sleep(duree_defilement)
    ActionChains(driver).key_up(Keys.ARROW_DOWN).perform()


def recolte_resultats():
    # Cible tous les liens de chaque et retourne une liste
    elements_resultats = driver.find_elements(by=By.XPATH, value="//div[@class='main-infos']/h2/a")
    
    liens_resultats = []

    for c in elements_resultats:
        liens_resultats.append(c.get_attribute("href"))

    return liens_resultats


def click_volet_telechargement():
    # Nécessaire pour ouvrir les options de téléchargement une fois la page du document ouverte
    bouton_choix_telechargement = driver.find_element(by=By.XPATH, value='//*[@id="pa-icon-download"]/span[2]')
    bouton_choix_telechargement.click()


def verif_bouton_txt():
    # Vérifie la présence de la checkbox texte et donc la présence d'une océrisation 
    statut_txt = False
    bouton_choix_txt = '//*[@id="idRadioButtonFormat_texteBrut"]'

    try:
        driver.find_element(by=By.XPATH, value=bouton_choix_txt)
        statut_txt = True
    except NoSuchElementException:
        statut_txt = False

    return statut_txt


def recolte_metadonnees():
    metadata = {
        "titre": "",
        "auteur": "",
        "date": "",
        "sujet": "",
        "texte": "",
    }

    bouton_telechargement = ("/html/body/main/div[2]/div[3]/div[5]/div/ul/li[3]/span/a/span[1]")

    # Ouverture du menu "en savoir plus" avec attente et collecte
    bouton_en_savoir_plus = driver.find_element(by=By.XPATH, value='//*[@id="acc_1-0_panel_2_trigger"]')
    bouton_en_savoir_plus.click()

    try:
        element = WebDriverWait(driver, 25).until(
            expected_conditions.presence_of_element_located(
                (By.XPATH, bouton_telechargement)
            )
        )
        
    finally:
        pass

    # Collecte de chaque métadonnée
    metadata["titre"] = extraction_champ("Titre")
    metadata["auteur"] = extraction_champ("Auteur")
    metadata["date"] = extraction_champ("Dated'édition")
    metadata["sujet"] = extraction_champ("Sujet")

    return metadata


def extraction_champ(nom_metadonnee):
    # Ciblage de l'élement contenant les métadonnées du document
    champs = driver.find_elements(by=By.XPATH, value='//*[@id="enSavoirPlusContent"]/div/div/div[1]/div[1]/div')

    i = 0
    emplacement_titre = 0

    # Comparaison entre l'étiquette du champ et la métadonnée voulue en supprimant auparavant les éventuels espaces
    for champ in champs:
        if (
            champ.get_attribute("textContent").replace(" ", "")[
                0 : len(nom_metadonnee)
            ]
            == nom_metadonnee
        ):

            # Si OK : comptage en vue de la selection de la bonne <div>
            emplacement_titre = i
            break

        i += 1
    
    # Récupération de l'élement via son "index" au sein de la succession de <div>
    metadata = driver.find_element(by=By.XPATH, value=
        f'//*[@id="enSavoirPlusContent"]/div/div/div[1]/div[1]/div[{emplacement_titre + 1}]/span[2]'
    )

    return metadata.get_attribute("textContent")


def recolte_texte(metadata):
    # Ouverture et collecte du texte intégral
    # Sélection de la checkbox pour récupération du texte océrisé brute
    bouton_choix_txt = driver.find_element(by=By.XPATH, value='//*[@id="idRadioButtonFormat_texteBrut"]')
    bouton_choix_txt.click()

    # Appel fonction defilement sur l'élement venant d'apparaître pour intéragir avec la checkbox "Conditions d'utilisation" le bouton "Télécharger"
    defilement("mCSB_3", 1)

    conditions_utilisation = driver.find_element(by=By.XPATH, value='//*[@id="checkboxes-0"]')
    conditions_utilisation.click()

    bouton_télécharger = driver.find_element(by=By.XPATH, value='//*[@id="idButtonLancerTelechargement"]')
    bouton_télécharger.click()

    # Switch vers l'onglet nouvellement ouvert contenant le fulltext
    driver.switch_to.window(driver.window_handles[-1])

    # Attente de "En savoir plus sur l'OCR" pour s'assurer que l'ensemble du texte est chargé
    try:
        element = WebDriverWait(driver, 25).until(
            expected_conditions.presence_of_all_elements_located((By.XPATH, "//p"))
        )
    finally:
        pass

    # Ajout du fulltext aux métadonnées précédemment consituées
    metadata["texte"] = driver.find_element(by=By.TAG_NAME, value="html").text

    # Fermeture de l'onglet
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

    return metadata


def exploitation_resultats(liens_resultats):    
    # Boucle d'exploitation de chacun des résultats à partir de la liste de liens
    for l in liens_resultats:
        driver.get(l)
        try:
            click_volet_telechargement()         
            if verif_bouton_txt() == True:
                dict_metadonnees = recolte_metadonnees()
                dict_metadonnees = recolte_texte(dict_metadonnees)
                time.sleep(1)
                sortie.append(dict_metadonnees)
            else:
                pass

            time.sleep(1)

        except:
            pass

In [3]:
sortie = []
fin_résultats = False
page = page_debut = 0 # 1ère page à exploiter -1

# Boucle défilement des pages appelant les fonctions de récoltes des résultats
while fin_résultats == False:
    page += 1
    driver.get(f"https://gallica.bnf.fr/services/engine/search/sru?operation=searchRetrieve&version=1.2&maximumRecords=15&collapsing=true&exactSearch=false&query=gallica%20all%20%22gymnastique%22%20%20and%20%28dc.language%20all%20%22fre%22%29%20and%20%28dc.type%20all%20%22monographie%22%20or%20dc.type%20all%20%22fascicule%22%29%20and%20%28gallicapublication_date%3E%3D%221848%22%20and%20gallicapublication_date%3C%3D%221914%22%29&filter=&page={page}")
    
    # Page 1 uniquement : agit sur le bandeau du consentement des cookies
    if page == 1:
        driver.implicitly_wait(3)
        refus_cookies = driver.find_element(by=By.ID, value="tarteaucitronAllDenied2")
        refus_cookies.click()

    # Arrêt en cas d'erreur     
    try:
        driver.find_element(by=By.CLASS_NAME, value="errorMessage")
        fin_résultats = True

    except:
        # Défilement de la page de résultat afin de charger l'ensemble du contenu dynamique
        defilement("mCSB_1", 6)
        
        # Appel des fonctions
        exploitation_resultats(recolte_resultats())
        
        # Condition d'arrêt à partir d'une certaine page
        if page == 20:
            break


In [4]:
# Export CSV
df = pd.DataFrame(sortie)
df.to_csv(f"data/gallica_p{page_debut}-{page}.csv")